In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7608\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Api
   #### base
    导出中间格式文件的函数，以onnx和torchsrtipt为目标
    export:主要是对torch.onnx.export函数的装饰，增加了模型以及函数的替换工作，就是mmdeploy的主要任务
    extract_partition：对给定模型进行切分的工作
    from_onnx:由onnx转化为后端，有些后端没有
    同样，在torchscript中也对torch.jit.trace函数进行了装饰，增加了一样的工作流程。

In [5]:
from mmdeploy.apis.onnx import export,extract_partition
from mmdeploy.apis.torch_jit import trace
model = create_model()
args = get_input_tensor()

#主函数
export(model, args,
    'place/to/save/model',
    backend='tensorrt',
    input_names=['input'],output_names=['output'],
    dynamic_axes={'input': {
    0: 'batch',
    2: 'height',
     3: 'width'
        }})

# backend 独立模块
   #### base
    后端管理器，需要功能 1. 将中间文件转化为后端文件，2.创建执行后端格式的模型

    1.BaseBackendManager继承后，实现to_backend 类方法（主函数），就是把中间文件转化为后端文件类型函数
    一般中间文件是onnx格式的，那就需要写一个from_onnx函数，将onnx文件转化为后端文件，并在这里调用  

    2.BaseWrapper是后端模型执行的抽象类，载入后端模型，为了让后端模型能够使用，需要加入forward函数的重写。

    最后，可能会有特有的后端库函数编译文件，也需要在这个包中实现加载。
   

In [ ]:
from mmdeploy.backend.base import BaseBackendManager,get_backend_manager,BaseWrapper
from mmdeploy.backend.onnxruntime import ORTWrapper
import torch
from mmengine.model import revert_sync_batchnorm

In [ ]:
onnx_file = 'model.onnx'
model = ORTWrapper(onnx_file, 'cpu')
inputs = dict(input=torch.randn(1, 3, 224, 224, device='cpu'))
outputs = model(inputs)
print(outputs)

# codebase 导入模块和创建Task（只需要注册继承）
   #### base文件--》import_codebase
     BaseBackendModel：后端模型的基类，这是一个中间接口，用来创建一个带有wrapper的后端模型
     MMCodebase:
         负责导入模型包，和创建Task类
            get_task_class：根据任务类型获得类
            build_task_processor：根据给定config,构建相应的任务Task类
            register_deploy_modules：（重写）
                import 该任务对应的models重写函数部分模块ie. mmdeploy.mmpretrain.models  
            register_all_modules：（重写）
                import 相应任务的所有模块包文件ie . mmpretrain
    BaseTask：
         根据给定的config,导入模型文件，并实例化模型，数据集等用于执行的组件，类似于runner
         有抽象方法需要实现。因为任务不同，所以构建组件的形式可能不同，需要重写
             
####  mm任务
    deploy：
        需要实现该任务对BaseTask ，MMCodebase和BaseBackendModel特定继承类
        MMCodebase:导入包，创建对应的Task实例
        BaseTask:整个任务的全流程
        BaseBackendModel：这个是对后端任务的模型的再包装，后端模型是需要让后端文件执行，
            但是还需要对输入进行预处理，这部分并不在模型中，所以需要外面处理好再给后端模型，
            所以这个模型是包括对输入的预处理和输出的预处理两个部分，同时本身还是一个nn.module
        
    models:
        from mmdeploy.core import FUNCTION_REWRITER

# core：
    这里就是mmdeploy替换模型的原理模块，使用替换的形式修改模型
      rewriter:
        RewriterRegistry：注册器
        RewriterManager：管理器
            分为function，module，symblic三个rewriter类
            FunctionRewriter：
                内部有个注册表，原理就是在执行模型的时候，将模型的函数，替换成设置的函数即可
            SymbolicRewriter：
                这个就是原来的Symbolic函数的写法
        RewriterContext，里面有一个RewriterManager管理器，用with 方式打开，后对function和symblic执行替换，而对于module，则要一个函数patch_model来执行替换
        
            @FUNCTION_REWRITER.register_rewriter(func_name='mmpretrain.models.backbones.shufflenet_v2.InvertedResidual.forward')
        def shufflenetv2_backbone__forward__default(self, x):
             return output

# mmcv
    这里是专门给MMCV一些函数的替换
# pytorch

      这里是一些需要替换的torch函数，用的都是

      @FUNCTION_REWRITER.register_rewriter(func_name='torch.Tensor.flatten', backend=Backend.NCNN.value)
      def myfunc():
          return tensor
      只有两种，一种是函数替换，一种是符号替换。

# 具体的调用过程

In [ ]:
在tool中有delpoy.py函数，需要给定deploy_config,model_config,checkpoint,img参数

主要做三项工作，torch2onnx（转onnx，或者torchscript）,extract_model(抽取模型),create_calib_input_data（量化数据集）

先转化为中间格式torch2onnx
torch2onnx：载入所有参数文件，用build_task_processor函数，实例化CodeBase类进行任务包的导入，然后调用MMCodebase.build_task_processor方法,
            创建一个任务处理器，那就可以用任务处理器创建所有的组件，比如创建输入，创建模型MODEL.build()。之后就会调用export函
            数，在函数中，需要完成对模型函数的替换工作（Core），最后调用torch.onnx.epxort生成onnx的文件。
        
再转化为后端文件
to_backend:就是将中间文件ONNX转化为对应后端文件的函数。创建BaseBackendManager调用BaseBackendManager里面的to_backend函数。
            不同框架的转化方式不同。
        
最后，可以对后端模型和pyorch模型进行结果可视化对比，这个过程也是用Task的完成的，需要BaseBackendManager让每个框架实现
带有forward函数的wrapper模型。

# 使用方法

In [ ]:
#toonnx
from mmdeploy.apis import torch2onnx
img = 'demo.jpg'
work_dir = 'work_dir'
save_file ='fcos.onnx'
deploy_cfg = ('configs/mmdet/detection/detection_onnxruntime_dynamic.py')
model_cfg = ('mmdetection/configs/fcos/fcos_r50_caffe_fpn_gn-head_1x_coco.py')
model_checkpoint = ('checkpoints/fcos_r50_caffe_fpn_gn-head_1x_coco-821213aa.pth')
device = 'cpu'

# from mmengine.registry import MODELS
#模型参数想要能使用下面的方法，必须是可以MODELS.build的类型，也就是说必须进行注册
torch2onnx(img,#图片
           work_dir,#保存路径
           save_file,#ir文件名
           deploy_cfg,
           model_cfg,
           model_checkpoint,
           device#  'cpu'
          )

In [ ]:
#tobackend，不同的后端可能会与onnx2backend这样的函数包装from_onnx函数
from mmdeploy.backend.tensorrt import from_onnx

from_onnx(
    onnx_path,
    output_prefix,
    input_shapes=final_params['input_shapes'],
    log_level=get_trt_log_level(),
    fp16_mode=final_params.get('fp16_mode', False),
    int8_mode=final_params.get('int8_mode', False),
    int8_param=int8_param,
    max_workspace_size=final_params.get('max_workspace_size', 0),
    device_id=device_id)

from mmdeploy.backend.onnxruntime import ONNXRuntimeManager

backendfile = ONNXRuntimeManager.to_backend(ir_files: Sequence[str],
                   work_dir: str,
                   log_level: int = logging.INFO,
                   device: str = 'cpu',)


#这只是后端模型，还需要对输入预处理才能正确使用，所以一般使用再装饰的模型
backendmodel = ONNXRuntimeManager.build_wrapper(backend_files)

In [ ]:
#这个东西很关键，直接给定参数，他会自动导入任务包，实例化出对应的模型
from mmdeploy.apis.utils import build_task_processor
task_processor = build_task_processor(model_cfg,#Config类
                                      deploy_cfg,#Config类
                                      device)


#这个就是后端模型
model = task_processor.build_backend_model(['/root/work_dir/FCOS.onnx'], 
                                        data_preprocessor_updater=task_processor.update_data_preprocessor)
input_shape = get_input_shape(deploy_cfg)
model_inputs, _ = task_processor.create_input(img, input_shape)
with torch.no_grad():
    result = model.test_step(model_inputs)[0]
    
task_processor.visualize(
    image=img,
    model=model,
    result=result,
    output_file='/root/test.jpg',
    window_name='test',
    show_result=True)

# 备注

In [ ]:
deploy_cfg
backend_config = dict(type='coreml', convert_to='mlprogram')#后端名称
onnx_config = dict(type='onnx',
    export_params=True,keep_initializers_as_inputs=False,
    opset_version=11, save_file='end2end.onnx',
                    input_names=['input'], 
                   output_names=['output'],
                   input_shape=None,#输入大小
                   optimize=True)#中间格式
codebase_config = dict(type='mmpretrain', task='Classification')#codebase的名称

# 检查模型是否正确

In [ ]:
from mmdeploy.apis import visualize_model
from mmdeploy.utils import Backend

model_cfg = 'mmdetection/configs/fcos/fcos_r50_caffe_fpn_gn-head_1x_coco.py'
deploy_cfg = 'configs/mmdet/detection/detection_onnxruntime_dynamic.py'
model = 'work_dir/fcos.onnx'
img = 'demo.jpg'
device = 'cpu'

In [ ]:
visualize_model(model_cfg,deploy_cfg,[checkpoint],img,'cpu',
                Backend.PYTORCH,
                show_result=True,output_file='/root/out.jpg')

In [ ]:
visualize_model(model_cfg,deploy_cfg,'/root/work_dir/FCOS.onnx',img,'cpu',
                Backend.ONNXRUNTIME,
                show_result=True,output_file='/root/out-onnx.jpg')

# 量化

In [ ]:
!git clone https://github.com/nihui/imagenet-sample-images --depth=1
python3 tools/deploy.py  configs/mmpretrain/classification_ncnn-int8_static.py 
${MODEL_CONFIG}  
${MODEL_PATH} 
/path/to/self-test.png  
--work-dir work_dir 
--device cpu 
--quant 
--quant-image-dir /path/to/imagenet-sample-images